In [1]:
import pandas as pd
import csv
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import math
# from scipy.spatial import distance
import nltk
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.display import display

# load csv data
eval_csv = pd.read_csv("../data/evaluation.csv")
item_csv = pd.read_csv("../data/items.csv", sep="|", quoting=3)
tran_csv = pd.read_csv("../data/transactions.csv", sep="|")

In [2]:
# check dataset
display(item_csv.head(5))

print("=====NA Counts=====")
display(item_csv.isnull().sum())

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]"
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]"


=====NA Counts=====


itemID           0
title            0
author        3247
publisher        9
main topic     259
subtopics        0
dtype: int64

In [3]:
# without duplicates dataframe
pd.set_option('display.max_rows', 20)
item_csv_no_duplicate = item_csv[~item_csv.duplicated(subset=['title',"author"])]
item_csv_no_duplicate.sort_values("author")

,itemID,title,author,publisher,main topic,subtopics
29573,43967,The Portal,"""Dc"" Hedlund",Westbow Press,FJ,[FL]
42562,30689,"''2046 (Back to 2046,Chinese Edition)",'',1 Plus Publishing & Consulting,DNT,[FL]
2011,46571,Triad Blood,'Nathan Burgoine,Bold Strokes Books,FM,[]
47587,9838,El Jaguar y El Aguila/The Jaguar and the Eagle,(Mixtli) Graycloud,WISE INK,YFP,[YFJ]
34388,38680,Can You Survive an Alien Invasion?: An Interac...,",Blake Hoena",Capstone Press,YFB,[]
...,...,...,...,...,...,...
78092,37942,Perry Rhodan 109. Das Loch im Universum,NaN,MOEWIG,FL,[]
78168,4022,Iron Man Armoured Avenger,NaN,Panini Publishing Ltd,XADC,[YF]
78173,30445,Find Out,NaN,Books on Demand,FM,[FL]
78180,45554,The Eyes of Despero!,NaN,GROSSET DUNLAP,YFZZ,[YFC]


In [4]:
# Example of Edit distance on topic
topic = item_csv_no_duplicate["main topic"]
print(topic[0],topic[3])
print(nltk.edit_distance(topic[0],topic[3]))

# also works on subtopics
print(nltk.edit_distance("5AP",["5AQ","5JA","5LF","5LKE","6FG","YFHR","YFM"],transpositions=True))

YFB YB
1
7


In [5]:
auth = item_csv_no_duplicate.set_index("author").loc["Victoria Aveyard"].set_index("itemID")
print("=============== Original Dataset ===============")
display(auth.head(10))

# find the close matching of book Goldener Käfig (Die Farben des Blutes 3) wiht main topic and sub topics [5AQ,5JA,5LF,5LKE,6FG,YFHR,YFM]
# within given dataframe
src_title     = "Goldener Käfig (Die Farben des Blutes 3)"
src_maintopic = "YFHR"
src_subtopics = ["5AQ","5JA","5LF","5LKE","6FG","YFHR","YFM"]

# note didn't remove duplicate but you should
auth.loc[:,"edit_score"] = 999

for i, row in tqdm(auth.iterrows(), desc = "Processing dataframe rows", total=len(auth)):
    # remove current book so it won't recommend same book
    if row['title'] == src_title:
        auth.loc[i, "edit_score"] = 999
        continue
    
    # subtopics
    tgt_subtopics = row['subtopics'].lstrip("[").rstrip("]").split(",")
    if tgt_subtopics == ['']:
        pass
    else:
        min_sub_score = math.inf
        for tgt_sub in tgt_subtopics:
            for src_sub in src_subtopics:
                score = nltk.edit_distance(tgt_sub,src_sub,transpositions=True)
                if score < min_sub_score:
                    min_sub_score = score
    # main topics
    tgt_maintopic = row['main topic']
    if tgt_maintopic == '':
        pass
    main_topic_score = nltk.edit_distance(src_maintopic,tgt_maintopic,transpositions=True)
    
    # assign edit score of sum
    auth.loc[i, "edit_score"] = min_sub_score + main_topic_score
    
print("=============== Recommendation ===============")
display(auth.sort_values("edit_score").head(100))

=============== Original Dataset ===============


,title,publisher,main topic,subtopics
itemID,,,,
19194,Red Queen 1,Orion Publishing Group,YFH,"[5AP,FBA]"
28585,Wütender Sturm (Die Farben des Blutes 4),Carlsen Verlag GmbH,YFHR,"[5AQ,YFM]"
7047,Die rote Königin (Die Farben des Blutes 1),Carlsen Verlag GmbH,YFH,"[5AQ,YFHR,YFM]"
42215,Gläsernes Schwert (Die Farben des Blutes 2),Carlsen Verlag GmbH,YFHR,"[5AQ,YFHR,YFM]"
50997,Broken Throne,Harper Collins Publ. USA,YFH,[5AP]
35354,Goldener Käfig (Die Farben des Blutes 3),Carlsen Verlag GmbH,YFHR,"[5AQ,5JA,5LF,5LKE,6FG,YFHR,YFM]"
38154,Red Queen 2-Book Box Set,Harper Collins Publ. USA,YFH,[5AP]
75459,Realm Breaker,Harper Collins Publ. USA,YFH,[]
40289,Red Queen 03. King's Cage,Orion Publishing Group,YFHR,"[5AN,FM]"


Processing dataframe rows:   0%|          | 0/15 [00:00<?, ?it/s]

=============== Recommendation ===============


,title,publisher,main topic,subtopics,edit_score
itemID,,,,,
28585,Wütender Sturm (Die Farben des Blutes 4),Carlsen Verlag GmbH,YFHR,"[5AQ,YFM]",0
42215,Gläsernes Schwert (Die Farben des Blutes 2),Carlsen Verlag GmbH,YFHR,"[5AQ,YFHR,YFM]",0
7047,Die rote Königin (Die Farben des Blutes 1),Carlsen Verlag GmbH,YFH,"[5AQ,YFHR,YFM]",1
40289,Red Queen 03. King's Cage,Orion Publishing Group,YFHR,"[5AN,FM]",1
35495,Red Queen 2. Glass Sword,Orion Publishing Group,YFHR,"[5AP,FBA]",1
19194,Red Queen 1,Orion Publishing Group,YFH,"[5AP,FBA]",2
50997,Broken Throne,Harper Collins Publ. USA,YFH,[5AP],2
38154,Red Queen 2-Book Box Set,Harper Collins Publ. USA,YFH,[5AP],2
75459,Realm Breaker,Harper Collins Publ. USA,YFH,[],2


In [6]:
#https://blog.csdn.net/zcr1024/article/details/88525969?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control

# target dataframe that we want to find matches
# note removed duplicates
auth = item_csv_no_duplicate.set_index("author").loc["Victoria Aveyard"].set_index("itemID")
print("=============== Original Dataset ===============")
display(auth.head(10))

# find the close matching of book Goldener Käfig (Die Farben des Blutes 3) with title
# within given dataframe
src_title     = "Goldener Käfig (Die Farben des Blutes 3)"
src_maintopic = "YFHR"
src_subtopics = ["5AQ","5JA","5LF","5LKE","6FG","YFHR","YFM"]
auth.loc[:,"fuzzy_match_score"] = 0

for i, row in tqdm(auth.iterrows(), desc = "Processing dataframe rows", total=len(auth)):
    # remove current book so it won't recommend same book
    if row['title'] == src_title:
        auth.loc[i, "fuzzy_match_score"] = 0
        continue
    
    # title fuzzy match
    tgt_title = row['title']
    if tgt_title == "":
        pass
    else:
        basic_ratio = fuzz.ratio(src_title, tgt_title)
        token_ratio = fuzz.token_sort_ratio(src_title, tgt_title)
        score = max([basic_ratio,token_ratio])
    
    # assign fuzzy max match score
    auth.loc[i, "fuzzy_match_score"] = score
    
print("=============== Recommendation ===============")
display(auth.sort_values("fuzzy_match_score", ascending=False).head(100))

print("=============== Method 2 (fast) ==============")

# method 2 without dataframe, should be faster
def MaxScorer(s1, s2):
    basic_ratio   = fuzz.ratio(s1, s2)
    token_ratio   = fuzz.token_sort_ratio(s1, s2)
    return max([basic_ratio,token_ratio])
recommend_list = process.extract(src_title, auth['title'], scorer=MaxScorer)
# duplication detection => score 0
result_no_dup = []
for rec in recommend_list:
    rec = list(rec)
    if rec[0] == src_title:
        # make score of itself to 0
        rec = tuple([rec[0], 0, rec[2]])
        result_no_dup.append(rec)
    else:
        result_no_dup.append(rec)
print(result_no_dup)

=============== Original Dataset ===============


,title,publisher,main topic,subtopics
itemID,,,,
19194,Red Queen 1,Orion Publishing Group,YFH,"[5AP,FBA]"
28585,Wütender Sturm (Die Farben des Blutes 4),Carlsen Verlag GmbH,YFHR,"[5AQ,YFM]"
7047,Die rote Königin (Die Farben des Blutes 1),Carlsen Verlag GmbH,YFH,"[5AQ,YFHR,YFM]"
42215,Gläsernes Schwert (Die Farben des Blutes 2),Carlsen Verlag GmbH,YFHR,"[5AQ,YFHR,YFM]"
50997,Broken Throne,Harper Collins Publ. USA,YFH,[5AP]
35354,Goldener Käfig (Die Farben des Blutes 3),Carlsen Verlag GmbH,YFHR,"[5AQ,5JA,5LF,5LKE,6FG,YFHR,YFM]"
38154,Red Queen 2-Book Box Set,Harper Collins Publ. USA,YFH,[5AP]
75459,Realm Breaker,Harper Collins Publ. USA,YFH,[]
40289,Red Queen 03. King's Cage,Orion Publishing Group,YFHR,"[5AN,FM]"


Processing dataframe rows:   0%|          | 0/15 [00:00<?, ?it/s]

=============== Recommendation ===============


,title,publisher,main topic,subtopics,fuzzy_match_score
itemID,,,,,
7047,Die rote Königin (Die Farben des Blutes 1),Carlsen Verlag GmbH,YFH,"[5AQ,YFHR,YFM]",76
28585,Wütender Sturm (Die Farben des Blutes 4),Carlsen Verlag GmbH,YFHR,"[5AQ,YFM]",75
42215,Gläsernes Schwert (Die Farben des Blutes 2),Carlsen Verlag GmbH,YFHR,"[5AQ,YFHR,YFM]",75
35495,Red Queen 2. Glass Sword,Orion Publishing Group,YFHR,"[5AP,FBA]",40
38135,Red Queen 4. War Storm,Harper Collins Publ. USA,YFH,[5AP],38
40289,Red Queen 03. King's Cage,Orion Publishing Group,YFHR,"[5AN,FM]",36
19194,Red Queen 1,Orion Publishing Group,YFH,"[5AP,FBA]",33
38154,Red Queen 2-Book Box Set,Harper Collins Publ. USA,YFH,[5AP],33
50997,Broken Throne,Harper Collins Publ. USA,YFH,[5AP],32


=============== Method 2 (fast) ==============
[('Goldener Käfig (Die Farben des Blutes 3)', 0, 35354), ['Die rote Königin (Die Farben des Blutes 1)', 78, 7047], ['Gläsernes Schwert (Die Farben des Blutes 2)', 75, 42215], ['Wütender Sturm (Die Farben des Blutes 4)', 74, 28585], ['Red Queen 2. Glass Sword', 40, 35495]]


# Time test for two methods (fuzzy title matches)

The first method iterates dataframe so it's slow

The second method uses buildin batch processes

Bonus: 

Time test of subtitle matches with edit distances (also slow since iter rows)

In [7]:
%%timeit

# target dataframe that we want to find matches
# note removed duplicates
auth = item_csv_no_duplicate.set_index("author").loc["Victoria Aveyard"].set_index("itemID")
# print("=============== Original Dataset ===============")
# display(auth.head(10))

# find the close matching of book Goldener Käfig (Die Farben des Blutes 3) with title
# within given dataframe
src_title     = "Goldener Käfig (Die Farben des Blutes 3)"
src_maintopic = "YFHR"
src_subtopics = ["5AQ","5JA","5LF","5LKE","6FG","YFHR","YFM"]
auth.loc[:,"fuzzy_match_score"] = 0

for i, row in auth.iterrows():
    # remove current book so it won't recommend same book
    if row['title'] == src_title:
        auth.loc[i, "fuzzy_match_score"] = 0
        continue
    
    # title fuzzy match
    tgt_title = row['title']
    if tgt_title == "":
        pass
    else:
        basic_ratio = fuzz.ratio(src_title, tgt_title)
        token_ratio = fuzz.token_sort_ratio(src_title, tgt_title)
        score = max([basic_ratio,token_ratio])
    
    # assign fuzzy max match score
    auth.loc[i, "fuzzy_match_score"] = score
    
# print("=============== Recommendation ===============")
# display(auth.sort_values("fuzzy_match_score", ascending=False).head(100))

14.6 ms ± 279 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
# method 2 without dataframe, should be faster
def MaxScorer(s1, s2):
    basic_ratio   = fuzz.ratio(s1, s2)
    token_ratio   = fuzz.token_sort_ratio(s1, s2)
    return max([basic_ratio,token_ratio])
recommend_list = process.extract(src_title, auth['title'], scorer=MaxScorer)
# duplication detection => score 0
result_no_dup = []
for rec in recommend_list:
    rec = list(rec)
    if rec[0] == src_title:
        # make score of itself to 0
        rec = tuple([rec[0], 0, rec[2]])
        result_no_dup.append(rec)
    else:
        result_no_dup.append(rec)
# print(result_no_dup)

270 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit

# Bonus
auth = item_csv_no_duplicate.set_index("author").loc["Victoria Aveyard"].set_index("itemID")
# print("=============== Original Dataset ===============")
# display(auth.head(10))

# find the close matching of book Goldener Käfig (Die Farben des Blutes 3) wiht main topic and sub topics [5AQ,5JA,5LF,5LKE,6FG,YFHR,YFM]
# within given dataframe
src_title     = "Goldener Käfig (Die Farben des Blutes 3)"
src_maintopic = "YFHR"
src_subtopics = ["5AQ","5JA","5LF","5LKE","6FG","YFHR","YFM"]

# note didn't remove duplicate but you should
auth.loc[:,"edit_score"] = 999

for i, row in auth.iterrows():
    # remove current book so it won't recommend same book
    if row['title'] == src_title:
        auth.loc[i, "edit_score"] = 999
        continue
    
    # subtopics
    tgt_subtopics = row['subtopics'].lstrip("[").rstrip("]").split(",")
    if tgt_subtopics == ['']:
        pass
    else:
        min_sub_score = math.inf
        for tgt_sub in tgt_subtopics:
            for src_sub in src_subtopics:
                score = nltk.edit_distance(tgt_sub,src_sub,transpositions=True)
                if score < min_sub_score:
                    min_sub_score = score
    # main topics
    tgt_maintopic = row['main topic']
    if tgt_maintopic == '':
        pass
    main_topic_score = nltk.edit_distance(src_maintopic,tgt_maintopic,transpositions=True)
    
    # assign edit score of sum
    auth.loc[i, "edit_score"] = min_sub_score + main_topic_score
    
# print("=============== Recommendation ===============")
# display(auth.sort_values("edit_score").head(100))